In [1]:
import nltk
from nltk.corpus import stopwords
import glob
import docx2txt
from nltk.probability import FreqDist
import math
from gensim.models.doc2vec import TaggedDocument
from gensim import models as mlls
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
stopwords_en=stopwords.words("english")
from docx import Document
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import difflib
import pandas as pd

C:\Users\VEERANI2\.conda\envs\ana_envi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def preprocessing(data):
    wordlist=nltk.word_tokenize(data)
    text=[w.lower() for w in wordlist if w not in stopwords_en]
    return text

# text = ''
bbc = preprocessing(docx2txt.process('buscrash_bbc.docx')) 
fox = preprocessing(docx2txt.process('buscrash_foxnews.docx'))

In [3]:
#TF-IDF Method
#TF Calculations
word_set=set(bbc).union(set(fox))
tf_freqd_bbc=FreqDist(bbc)
bbc_length=len(bbc)
tf_bbc_tf_dict=dict.fromkeys(word_set,0)
for word in bbc:
    tf_bbc_tf_dict[word] = tf_freqd_bbc[word]/bbc_length
    
tf_freqd_fox=FreqDist(fox)
fox_length=len(fox)
tf_fox_tf_dict=dict.fromkeys(word_set,0)
for word in fox:
    tf_fox_tf_dict[word] = tf_freqd_fox[word]/fox_length

In [4]:
#IDF Calculations
text_idf_dict=dict.fromkeys(word_set,0)
text_length=2 #2 docs
for word in text_idf_dict.keys():
    if word in bbc:
        text_idf_dict[word]+=1
    if word in fox:
        text_idf_dict[word]+=1
        
for word,val in text_idf_dict.items():
    text_idf_dict[word] =1+math.log(text_length/(float(val)))

In [5]:
#TF-IDF Calcculations
#TFIDF of a word = (TF of the word) * (IDF of the word)
bbc_tfidf_dict=dict.fromkeys(word_set,0)
for word in bbc:
    bbc_tfidf_dict[word]=(tf_bbc_tf_dict[word])*(text_idf_dict[word])
    
fox_tfidf_dict=dict.fromkeys(word_set,0)
for word in fox:
    fox_tfidf_dict[word]=(tf_fox_tf_dict[word])*(text_idf_dict[word])

In [6]:
#Once the words in the text are vectorised, the similarity score between them is nothing but the ‘distance’ between them.

In [7]:
#Distance Computation
#Cosine Distance
v1=list(bbc_tfidf_dict.values())
v2=list(fox_tfidf_dict.values())
similarity=1-nltk.cluster.cosine_distance(v1,v2)
print("Similarity Index: {:4.2f}%".format(similarity*100))

Similarity Index: 71.20%


In [8]:
def sentfromdoc(filename):
    data=Document(filename)
    list1=list()
    list2=list()
    for p in data.paragraphs:
        list1.append(p.text)
    for val in list1:
        tokens = nltk.sent_tokenize(val)
        for t in tokens:
            list2.append(t)
    return list2
    
# bbc1 = Document('buscrash_bbc.docx')
# fox1 = Document('buscrash_foxnews.docx')

l_bbc=sentfromdoc('buscrash_bbc.docx')
l_fox=sentfromdoc('buscrash_foxnews.docx')

In [9]:
# Find and print the diff:
df=open('Output.txt','w')
df.truncate(0)
for line in difflib.unified_diff(
        l_bbc, l_fox, fromfile='buscrash_bbc.docx', 
        tofile='buscrash_foxnews.docx', lineterm=''):
        df.write(line)
        df.write('\n')
df.close()

In [10]:
file = open(r"Output.txt","r")

In [11]:
list1=list()
list2=list()
list3=list()
for l in file.readlines():
    if l.startswith('+'):
        list1.append(l)
    elif l.startswith('-'):
        list2.append(l)
    elif l.startswith('@'):
        pass;
    else:
        list3.append(l)

In [12]:
list4= [None] * max(len(list1),len(list2),len(list3))
list5=[None] * max(len(list1),len(list2),len(list3))
list6 =[None] * max(len(list1),len(list2),len(list3))

In [13]:
list4[:len(list1)]=list1
list5[:len(list2)]=list2
list6[:len(list3)]=list3

In [24]:
df = pd.DataFrame(np.column_stack([list4, list5, list6]), 
                               columns=['only_1', 'only_2', 'common'])
df['only_1']=df['only_1'].map(lambda x: x and x.lstrip('+%=/-#$;"!.\(!\&=&:%;'))
df['only_2']=df['only_2'].map(lambda x: x and x.lstrip('+%=/-#$;"!\.(!\&=&:%;'))
df['common']=df['common'].map(lambda x: x and x.lstrip('+%=/-#$;"!\(.!\&=&:%;'))

In [25]:
df

,only_1,only_2,common
0,buscrash_foxnews.docx\n,buscrash_bbc.docx\n,"The bus, registered in North Macedonia, crash..."
1,A bus crash in early Tuesday has killed at lea...,"At least 46 people, including 12 children, die...",Seven people were taken to hospitals for trea...
2,The cause of the crash was not immediately con...,The bus was registered in North Macedonia and ...,Part of the motorway barrier was destroyed in...
3,Officials said an investigation will be launch...,It rammed a crash barrier on a motorway south-...,Bulgaria's interim Prime Minister Stefan Yane...
4,Bulgarian news agency Novinite said representa...,Seven people escaped from the bus and were tak...,None
5,Photos taken shortly after the crash showed th...,Four-year-old twin boys were among those kille...,None
6,Bulgarian Caretaker Prime Minister Stefan Yane...,Bulgarian Interior Minister Boyko Rashkov visi...,None
7,I take this opportunity to send my condolences...,The cause of the disaster was not immediately ...,None
8,Let’s hope we learn lessons from this tragic i...,Bulgarian officials described how the bus swer...,None
9,North Macedonian Prime Minister Zoran Zaev tol...,Pictures of the aftermath showed a section of ...,None
